<a href="https://colab.research.google.com/github/123san-art/Printique/blob/main/firstversion_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================
# ✅ CELL 1: ENVIRONMENT SETUP (Persistent with Google Drive)
# ==============================================================

from google.colab import drive
drive.mount('/content/drive')

# --- Persistent paths ---
DRIVE_CACHE = "/content/drive/MyDrive/printique_cache"
SHAPE_REPO = f"{DRIVE_CACHE}/shap-e"
SHAPE_MODEL_CACHE = f"{DRIVE_CACHE}/shap_e_weights"

import os, sys, pathlib, subprocess

# --- System dependencies ---
!apt-get update -qq && apt-get install -y -qq git-lfs unzip cura-engine
!git lfs install

# --- Clone Shap-E into Drive cache (first time only) ---
if not os.path.exists(SHAPE_REPO):
    print("⏬ Cloning Shap-E into Drive for persistence...")
    !git clone https://github.com/openai/shap-e.git $SHAPE_REPO
else:
    print("✅ Shap-E repo found in Drive cache")

# Link repo into runtime
!ln -sf $SHAPE_REPO /content/shap-e
if "/content/shap-e" not in sys.path:
    sys.path.append("/content/shap-e")

# --- Python dependencies (conflict-free) ---
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q flask flask-cors pyngrok trimesh open3d pillow numpy tqdm matplotlib

# --- Verify Cura ---
cura_path = "/usr/bin/CuraEngine"
if pathlib.Path(cura_path).exists():
    print("✅ CuraEngine installed.")
else:
    print("⚠️ CuraEngine missing, run: !apt-get install -y cura-engine")

# --- GPU check ---
import torch
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)} (CUDA {torch.version.cuda})")
else:
    print("⚠️ No GPU detected — using CPU.")

# --- Install Shap-E locally ---
!pip install -e $SHAPE_REPO -q

# --- Model cache path ---
os.makedirs(SHAPE_MODEL_CACHE, exist_ok=True)
os.environ["SHAPE_CACHE_DIR"] = SHAPE_MODEL_CACHE

print("\n✅ Environment setup complete (cached in Drive).")

Mounted at /content/drive
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libarcus3:amd64.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../libarcus3_4.13.0-2build1_amd64.deb ...
Unpacking libarcus3:amd64 (4.13.0-2build1) ...
Selecting previously unselected package libpolyclipping22:amd64.
Preparing to unpack .../libpolyclipping22_6.4.2-7_amd64.deb ...
Unpacking libpolyclipping22:amd64 (6.4.2-7) ...
Selecting previously unselected package cura-engine.
Preparing to unpack .../cura-engine_1%3a4.13.0-1_amd64.deb ...
Unpacking cura-engine (1:4.13.0-1) ...
Setting up libpolyclipping22:amd64 (6.4.2-7) ...
Setting up libarcus3:amd64 (4.13.0-2build1) ...
Setting up cura-engine (1:4.13.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-b

In [2]:
# ==============================================================
# ✅ CELL 2: FRONTEND RESTORE / SYNC (Persistent via Drive)
# ==============================================================

from google.colab import drive
drive.mount('/content/drive')

import os, shutil

FRONTEND_DRIVE_DIR = "/content/drive/MyDrive/Printique3DFrontend"
LOCAL_FRONTEND_DIR = "/content/frontend"

# Create local directory if not present
os.makedirs(LOCAL_FRONTEND_DIR, exist_ok=True)

# --------------------------------------------------------------
# If not cached in Drive, ask user to upload ZIP once
# --------------------------------------------------------------
if not os.path.exists(FRONTEND_DRIVE_DIR):
    print("⏬ Drive folder not found. Please upload your frontend zip file (printique_frontend.zip).")
    from google.colab import files
    uploaded = files.upload()

    if 'printique_frontend.zip' in uploaded:
        print("📦 Unzipping frontend to local folder...")
        !unzip -o printique_frontend.zip -d /content/frontend
        print("📂 Copying to Drive cache...")
        shutil.copytree(LOCAL_FRONTEND_DIR, FRONTEND_DRIVE_DIR, dirs_exist_ok=True)
        print("✅ Frontend successfully cached in Drive.")
    else:
        print("❌ Please make sure the zip file is named 'printique_frontend.zip'.")
else:
    print("♻️ Restoring frontend from Drive cache...")
    shutil.copytree(FRONTEND_DRIVE_DIR, LOCAL_FRONTEND_DIR, dirs_exist_ok=True)

print("✅ Frontend ready at:", LOCAL_FRONTEND_DIR)
!ls $LOCAL_FRONTEND_DIR


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
♻️ Restoring frontend from Drive cache...
✅ Frontend ready at: /content/frontend
requirements.txt  shap_e_model_cache  static  templates


In [3]:
# ==============================================================
# ✅ CELL 2.5: REFRESH FRONTEND FROM DRIVE (No zip needed)
# ==============================================================

import os, shutil

FRONTEND_FOLDER = "/content/drive/MyDrive/Printique3DFrontend"
LOCAL_FRONTEND_DIR = "/content/frontend"

# --- Check if Drive folder exists ---
if not os.path.exists(FRONTEND_FOLDER):
    raise FileNotFoundError(f"❌ Drive folder not found: {FRONTEND_FOLDER}\nPlease verify that it exists in Google Drive.")

# --- Remove old local frontend if it exists ---
if os.path.exists(LOCAL_FRONTEND_DIR):
    shutil.rmtree(LOCAL_FRONTEND_DIR)
    print(f"🧹 Old local frontend removed: {LOCAL_FRONTEND_DIR}")

# --- Copy fresh frontend from Drive to local Colab folder ---
shutil.copytree(FRONTEND_FOLDER, LOCAL_FRONTEND_DIR)
print(f"✅ Frontend copied from Drive to local folder: {LOCAL_FRONTEND_DIR}")

# --- Verify the structure ---
!ls -R $LOCAL_FRONTEND_DIR | head -n 40


🧹 Old local frontend removed: /content/frontend
✅ Frontend copied from Drive to local folder: /content/frontend
/content/frontend:
requirements.txt
shap_e_model_cache
static
templates

/content/frontend/shap_e_model_cache:
transmitter_config.yaml

/content/frontend/static:
css
js

/content/frontend/static/css:
style.css

/content/frontend/static/js:
script.js

/content/frontend/templates:
index.html


In [4]:
# ==============================================================
# ✅ CELL 3: AUTO-RECOVERY for Shap-E import path
# ==============================================================

import sys, os
repo_path = "/content/shap-e"

if not os.path.exists(repo_path):
    raise SystemExit("❌ Shap-E repo missing — please rerun Cell 1.")
if repo_path not in sys.path:
    sys.path.append(repo_path)
    print("✅ Shap-E import path restored.")
else:
    print("✅ Shap-E already active in sys.path.")


✅ Shap-E already active in sys.path.


In [ ]:
# ==============================================================
# ✅ CELL 4: LOAD SHAP-E MODELS
# ==============================================================

import torch
from shap_e.models.download import load_model, load_config
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("🚀 Loading Shap-E models ...")
try:
    xm = load_model("transmitter", device=device)
    model_text = load_model("text300M", device=device)
    model_image = load_model("image300M", device=device)
    diffusion = diffusion_from_config(load_config("diffusion"))
    print("✅ All Shap-E models loaded successfully.")
except Exception as e:
    print("❌ Error loading models:", e)


🚀 Loading Shap-E models ...


/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 89.6MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
# ==============================================================
# ✅ CELL 5 (Optimized): Generate Variants but Save Later
# ==============================================================

!pip install open3d -q
from PIL import Image, ImageEnhance, ImageOps
from google.colab import files
import trimesh, numpy as np, time, json
from shap_e.diffusion.sample import sample_latents
from shap_e.util.notebooks import decode_latent_mesh
from pathlib import Path
import torch

OUT_DIR = Path("/content/outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def preprocess_image_for_shape(image_path, save_path="/content/preprocessed_input.png"):
    img = Image.open(image_path).convert("RGB")
    w, h = img.size; s = min(w, h)
    left, top = (w - s)/2, (h - s)/2
    img = img.crop((left, top, left+s, top+s)).resize((512, 512), Image.LANCZOS)
    img = ImageEnhance.Contrast(img).enhance(1.3)
    img = ImageEnhance.Brightness(img).enhance(1.1)
    img = ImageEnhance.Color(img).enhance(1.2)
    img = ImageOps.equalize(img)
    img.save(save_path)
    return save_path

print("Choose input type:\n1. Text prompt\n2. Hybrid (Text + Image)")
choice = input("Enter 1 or 2: ").strip()

if choice == "1":
    prompt = input("Enter text prompt: ").strip() or "a simple object"
    model_to_use = model_text
    model_kwargs = dict(texts=[prompt])
elif choice == "2":
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
    processed_path = preprocess_image_for_shape(image_path)
    pil_image = Image.open(processed_path).convert("RGB")
    prompt = input("Enter supporting text prompt (optional): ").strip() or "a 3D object based on this image"
    model_to_use = model_image
    model_kwargs = dict(images=[pil_image], texts=[prompt])
else:
    raise SystemExit("Invalid input choice.")

print("\n🚀 Generating 2 preview variants (no .stl export yet)...")
start = time.time()

mesh_variants = []
for i in range(2):
    latents = sample_latents(
        batch_size=1,
        model=model_to_use,
        diffusion=diffusion,
        guidance_scale=15.0,
        model_kwargs=model_kwargs,
        progress=False,
        clip_denoised=True,
        use_fp16=torch.cuda.is_available(),
        use_karras=True,
        karras_steps=192,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0
    )

    latent = latents[0]
    tri_mesh = decode_latent_mesh(xm, latent).tri_mesh()
    mesh = trimesh.Trimesh(vertices=tri_mesh.verts, faces=tri_mesh.faces, process=False)
    mesh.fix_normals()

    ply_path = OUT_DIR / f"variant_{i+1}.ply"
    mesh.export(ply_path)  # only lightweight preview
    mesh_variants.append(str(ply_path))

    print(f"✅ Generated variant_{i+1}.ply (preview only)")

print(f"\n✅ 2 variants generated in {(time.time()-start):.2f}s.")
print("👉 The final .stl will be created only after you select a variant in the web app.")


Choose input type:
1. Text prompt
2. Hybrid (Text + Image)
Enter 1 or 2: 1
Enter text prompt: chair

🚀 Generating 2 preview variants (no .stl export yet)...


In [14]:
# ==============================================================
# ✅ CELL 6: USER SELECTION + AUTO-REPAIR + AUTO-SCALE + STL EXPORT
# ==============================================================

import trimesh, json, open3d as o3d, numpy as np
from pathlib import Path

OUT_DIR = Path("/content/outputs")
ply_variants = sorted(OUT_DIR.glob("variant_*.ply"))

if not ply_variants:
    raise SystemExit("❌ No PLY variants available. Please run Cell 5 first.")

print("Available variants:")
for i, f in enumerate(ply_variants, 1):
    print(f"{i}. {f.name}")

# --- User selects preferred model ---
try:
    choice = int(input("\nSelect a variant number to finalize and export as STL: ").strip())
    selected_ply = ply_variants[choice - 1]
except Exception:
    print("⚠️ Invalid input — selecting the latest by default.")
    selected_ply = ply_variants[-1]

# --- Load selected mesh ---
mesh = trimesh.load_mesh(selected_ply)
print(f"✅ Loaded mesh: {selected_ply.name}")
print(f"   Volume before scaling: {mesh.volume:.2f} mm³ | Faces: {len(mesh.faces)} | Watertight: {mesh.is_watertight}")

# ==============================================================
# 🧠 ADVANCED AUTO-REPAIR
# ==============================================================

mesh.update_faces(mesh.nondegenerate_faces())
mesh.remove_unreferenced_vertices()
mesh.remove_degenerate_faces()
mesh.fix_normals()
trimesh.repair.fill_holes(mesh)

# --- Poisson reconstruction for watertightness if needed ---
if not mesh.is_watertight:
    try:
        print("🧩 Running Poisson reconstruction for watertightness...")
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(mesh.vertices)
        pcd.estimate_normals()
        mesh_o3d, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)
        densities = np.asarray(densities)
        keep_mask = densities > np.quantile(densities, 0.05)
        mesh_o3d.remove_vertices_by_mask(~keep_mask)
        mesh = trimesh.Trimesh(
            vertices=np.asarray(mesh_o3d.vertices),
            faces=np.asarray(mesh_o3d.triangles),
            process=False
        )
        mesh.fix_normals()
        print("✅ Poisson reconstruction complete + artifacts removed.")
    except Exception as e:
        print(f"⚠️ Poisson reconstruction failed: {e}. Using fallback repair.")
        trimesh.repair.fill_holes(mesh)
else:
    print("✅ Mesh already watertight; skipping reconstruction.")

# ==============================================================
# 📏 AUTO-SCALE to keychain / print-ready size
# ==============================================================

min_realistic_volume = 10_000     # mm³  (~10 cm³)
target_volume = 50_000            # mm³  (~50 cm³ ≈ small keychain)

if mesh.volume < min_realistic_volume:
    scale_factor = (target_volume / (mesh.volume or 1)) ** (1/3)
    mesh.apply_scale(scale_factor)
    print(f"⚙️ Auto-scaled mesh by {scale_factor:.2f}× to reach printable keychain size (~5 cm).")
else:
    print("📏 Mesh volume within realistic printable range — no scaling applied.")

# ==============================================================
# ✨ FINAL SURFACE SMOOTHING
# ==============================================================

try:
    mesh = trimesh.smoothing.filter_laplacian(mesh, iterations=3)
    print("✨ Surface smoothing applied for better STL quality.")
except Exception:
    print("⚠️ Smoothing skipped due to instability.")

print(f"✅ Final Mesh → Faces: {len(mesh.faces)} | Watertight: {mesh.is_watertight} | Volume: {mesh.volume:.2f} mm³")

# ==============================================================
# 💾 EXPORT STL + SAVE METADATA
# ==============================================================

final_stl_path = OUT_DIR / f"{selected_ply.stem}_final.stl"
mesh.export(final_stl_path)

metadata = {
    "selected_model": selected_ply.name,
    "stl_path": str(final_stl_path),
    "faces": len(mesh.faces),
    "vertices": len(mesh.vertices),
    "watertight": mesh.is_watertight,
    "volume_mm3": float(mesh.volume),
}

meta_path = OUT_DIR / "selection_metadata.json"
with open(meta_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"\n💾 STL exported → {final_stl_path}")
print(f"🧾 Metadata saved → {meta_path}")

# Pass to next cell (analytics)
latest_stl = final_stl_path
final_mesh = mesh

print("\n📡 Ready for Cell 7: Print analytics + filament/time estimation.")


SystemExit: ❌ Not enough variants found. Run Cell 5 first.

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# ==============================================================
# ✅ CELL 7: SMART PRINT ANALYTICS (with Layer Height)
# ==============================================================

import trimesh, math, matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown
from pathlib import Path

OUT_DIR = Path("/content/outputs")
latest_stl = sorted(OUT_DIR.glob("*_final.stl"))[-1]
mesh = trimesh.load_mesh(latest_stl)

materials = {
    "PLA":  {"density": 1.24, "speed": 50},
    "ABS":  {"density": 1.04, "speed": 40},
    "PETG": {"density": 1.27, "speed": 35},
    "TPU":  {"density": 1.20, "speed": 25}
}

def estimate(material="PLA", infill=20.0, scale=1.0, layer_height=0.2):
    props = materials[material]

    # --- Scaled volume ---
    vol = mesh.volume * (scale**3)
    filament_diam = 1.75
    f_area = math.pi * (filament_diam / 2)**2
    f_len = (vol / f_area) / 1000
    mass = (vol / 1000) * props["density"]

    # --- Print time estimation ---
    # proportional to layer height (smaller layer = more layers = longer time)
    extrusion_width = 0.4
    speed = props["speed"]
    extrusion_rate = speed * extrusion_width * layer_height
    extrusion_rate_hr = extrusion_rate * 3600
    hrs = (vol / extrusion_rate_hr) * 1.25
    hrs_adj = hrs * (0.2 / layer_height)  # adjust for smaller/larger layers

    # --- Display results ---
    print(f"📦 Volume: {vol/1000:.2f} cm³ | Filament: {f_len:.2f} m | "
          f"Weight: {mass:.2f} g | Time: {hrs_adj:.1f} hr | "
          f"Layer: {layer_height:.2f} mm")

    # --- Visualization ---
    plt.figure(figsize=(5.5, 3))
    labels = ["Shell", "Infill", "Top/Bottom"]
    values = [vol * 0.1, vol * (infill / 100), vol * 0.05]
    plt.bar(labels, [v / 1000 for v in values],
            color=["#5DADE2", "#58D68D", "#F5B041"])
    plt.ylabel("Volume (cm³)")
    plt.title(f"{material} Volume Composition")
    plt.show()

# --- Interactive dashboard ---
interact(
    estimate,
    material=Dropdown(options=list(materials.keys()), value="PLA", description="Material:"),
    infill=FloatSlider(value=20, min=0, max=100, step=5, description="Infill (%)"),
    scale=FloatSlider(value=1.0, min=0.1, max=5, step=0.1, description="Scale ×"),
    layer_height=FloatSlider(value=0.2, min=0.05, max=0.4, step=0.05, description="Layer (mm)")
)


In [ ]:
import os
out_path = "/content/outputs"
print("Files inside /content/outputs:")
for f in os.listdir(out_path):
    print("  ", f)


In [ ]:
!pip install flask flask-cors pyngrok shap-e trimesh open3d -q

In [ ]:
# ==============================================================
# 🚀 PRINTIQUE — STABLE AUTO-LAUNCH (Flask + Ngrok FIXED)
# ==============================================================

import os, shutil, zipfile, time
from google.colab import drive
from pyngrok import ngrok

# -------- 1️⃣ MOUNT DRIVE --------
drive.mount('/content/drive')
DRIVE_ZIP = "/content/drive/MyDrive/printique_full_frontend.zip"
WORK_DIR = "/content/printique_full_frontend"

# -------- 2️⃣ RESTORE OR EXTRACT APP --------
if not os.path.exists(WORK_DIR):
    print("⏬ Extracting Printique app from Drive...")
    os.makedirs(WORK_DIR, exist_ok=True)
    with zipfile.ZipFile(DRIVE_ZIP, 'r') as z:
        z.extractall(WORK_DIR)
    print("✅ App extracted to", WORK_DIR)
else:
    print("♻️ Using cached app in:", WORK_DIR)

# -------- 3️⃣ INSTALL DEPENDENCIES --------
print("📦 Installing required dependencies...")
!pip install flask flask-cors pyngrok shap-e trimesh open3d -q

# -------- 4️⃣ AUTHENTICATE NGROK --------
ngrok.set_auth_token("34I4kTNwp3t9RFwEFx5baC0SrOQ_5xQaYqkcdSzRtMwz9oDvR")  # 👈 paste your Ngrok token inside quotes
print("✅ Ngrok authenticated successfully.")

# -------- 5️⃣ START FLASK (Foreground)+ NGROK --------

print("🚀 Starting Flask app...")
get_ipython().system_raw("python app.py &")  # run Flask in background but keep it alive

time.sleep(8)  # give Flask time to start

# -------- 6️⃣ START NGROK TUNNEL --------
public_url = ngrok.connect(5000)
print("🌍 Your Printique app is live at:", public_url)
print("\n💡 Tip: Leave this cell running. Open the link above in a new tab.")
